# Routing Chains to direct questions to prompts


## Router Chains using LCEL

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

For e.g., Say we have four prompts optimized for different types of questions, and we want to choose the prompt template based on the user input.

There are two ways to perform routing:

1. Using a `RunnableBranch`
2. Writing custom factory function that takes the input of a previous step and returns a **runnable**. Importantly, this should return a **runnable** and NOT actually execute.

We'll illustrate both methods using a two step sequence where the first step classifies an input question as being about Math, Physics, Computer Science or History, then routes to a corresponding prompt chain.

We also need to define a *General* chain as a fall back mechanism.

Here, we create a simple `classifier_template` that classifies the input text into one of the above 4 categories.

In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/utils")

In [2]:
import re
import warnings
from pathlib import Path
from types import FunctionType

import boto3
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT
from IPython.display import Markdown
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableBranch, RunnableLambda
from rich import print
from utils import get_inference_parameters

warnings.filterwarnings("ignore")

%load_ext rich
%load_ext autoreload
%autoreload 2

### Initialize Bedrock LLM

Here we utilize `langchain.llms.bedrock.Bedrock` class to initialize our LLM.

In [3]:
from langchain.llms.bedrock import Bedrock

region = "us-west-2"

# get bedrock runtime client
client = boto3.client("bedrock-runtime", region_name=region)
cv2_model_id = "anthropic.claude-v2"  # Bedrock model_id
instant_model_id = "anthropic.claude-instant-v1"
model_kwargs = get_inference_parameters("anthropic")  # Model kwargs for Anthropic LLMs
# print(model_kwargs)

llm_cv2 = Bedrock(
    client=client,
    model_id=cv2_model_id,
    model_kwargs=model_kwargs,
    region_name=region,
)  # Initalize LLM with Claude-v2

llm_instant = Bedrock(
    client=client,
    model_id=instant_model_id,
    model_kwargs=model_kwargs,
    region_name=region,
)  # Initalize LLM with Claude-Instant

### Create a Classifier prompt

Classifier prompt classifies the question into one of the following categories
- math
- physics
- computerscience
- history

In [4]:
from langchain.llms.anthropic import Anthropic
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough

# llm = bedrock_model

classifier_template = """Given the user question below, classify it as either being about `math`, `physics`, `computerscience` or `history`.

Do not respond with more than one word.

<question> {question} </question>

Classification:"""

classifier_template = f"{HUMAN_PROMPT}{classifier_template}{AI_PROMPT}"

classify_chain = (
    {"question": RunnablePassthrough()}
    | PromptTemplate.from_template(template=classifier_template)
    | llm_instant  # Here we use Claude-Instant for quicker response
    | StrOutputParser()
)

# test classifier chain
# classify_chain.invoke("List last 5 presidents of USA and their term durations.")

### Create separate prompts and chains to handle questions accordingly

physics, history, math and computerscience

In [5]:
# Create a directory to save prompts to text files
PROMPT_DIR = Path("./prompts")
PROMPT_DIR.mkdir(exist_ok=True)

##### Physics chain to answer physics questions

In [6]:
prompt_path = PROMPT_DIR.joinpath("physics_prompt_claude.txt")

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a <question> you admit\
that you don't know.

<question> {question} </question>

Answer:"""

# Format prompt for Anthropic
physics_prompt = PromptTemplate.from_template(
    template=f"{HUMAN_PROMPT}{physics_template}{AI_PROMPT}"
)

# save prompt to disk
prompt_path.write_text(physics_prompt.template, encoding="utf-8")

physics_chain = {"question": RunnablePassthrough()} | physics_prompt | llm_cv2

In [7]:
prompt_path = PROMPT_DIR.joinpath("math_prompt_claude.txt")

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

<question>
{question}
</question>

Answer:"""

# Format prompt for Anthropic
math_prompt = PromptTemplate.from_template(
    template=f"{HUMAN_PROMPT}{math_template}{AI_PROMPT}"
)

# save prompt to disk
prompt_path.write_text(math_prompt.template, encoding="utf-8")

math_chain = {"question": RunnablePassthrough()} | math_prompt | llm_cv2

In [8]:
prompt_path = PROMPT_DIR.joinpath("history_prompt_claude.txt")

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

<question>
{question}
</question>

Answer:"""

# Format prompt for Anthropic
history_prompt = PromptTemplate.from_template(
    template=f"{HUMAN_PROMPT}{history_template}{AI_PROMPT}"
)

# save prompt to disk
prompt_path.write_text(history_prompt.template, encoding="utf-8")

history_chain = {"question": RunnablePassthrough()} | history_prompt | llm_cv2

In [9]:
prompt_path = PROMPT_DIR.joinpath("computerscience_prompt_claude.txt")

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

<question>
{question}
</question>

Answer:"""

# Format prompt for Anthropic
computerscience_prompt = PromptTemplate.from_template(
    template=f"{HUMAN_PROMPT}{computerscience_template}{AI_PROMPT}"
)

# save prompt to disk
prompt_path.write_text(computerscience_prompt.template, encoding="utf-8")

computerscience_chain = (
    {"question": RunnablePassthrough()} | computerscience_prompt | llm_cv2
)

In [10]:
# General chain for fallback
general_template = (
    """Respond to the following question: <question>{question}</question> Answer:"""
)
general_prompt = PromptTemplate.from_template(
    template=f"{HUMAN_PROMPT}{general_template}{AI_PROMPT}"
)

general_chain = {"question": RunnablePassthrough()} | general_prompt | llm_cv2

In [11]:
# from pathlib import Path

# PROMPT_DIR = Path("./prompts")
# physics_prompt_path = PROMPT_DIR.joinpath("physics_prompt_claude.txt")
# _ = physics_prompt_path.write_text(
#     f"{HUMAN_PROMPT}{physics_template}{AI_PROMPT}", encoding="utf-8"
# )

# math_prompt_path = PROMPT_DIR.joinpath("math_prompt_claude.txt")
# _ = math_prompt_path.write_text(
#     f"{HUMAN_PROMPT}{math_template}{AI_PROMPT}", encoding="utf-8"
# )

# history_prompt_path = PROMPT_DIR.joinpath("history_prompt_claude.txt")
# _ = history_prompt_path.write_text(
#     f"{HUMAN_PROMPT}{history_template}{AI_PROMPT}", encoding="utf-8"
# )

### Putting it all together.

First, we pass the input **question** to a chain that categorizes, the **question** into one of 4 categories:

- Math
- Physics
- Computer Science
- History

Next, we pass the output to the `RunnableBranch`, which then, routes the **question** to the relevant *chain*

#### Using a RunnableBranch

A `RunnableBranch` is initialized with a list of (condition, runnable) pairs and a default runnable.

It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.

If no provided conditions match, it runs the default runnable.

Create a `RunnableBranch` that routes the input to the appropriate prompt.

In [12]:
prompt_branch = RunnableBranch(
    (lambda x: "math" in x["topic"].lower(), math_chain),
    (lambda x: "physics" in x["topic"].lower(), physics_chain),
    (lambda x: "history" in x["topic"].lower(), history_chain),
    (lambda x: "computerscience" in x["topic"].lower(), computerscience_chain),
    general_chain,
)

In [13]:
full_chain = {
    "topic": classify_chain,
    "question": RunnablePassthrough(),
} | prompt_branch

### Run the chain with questions from 4 topics

In [14]:
questions = [
    "List last 5 presidents of USA and their term durations. Output in a Markdown table format.",
    "What is 5% of 257",
    "What are black holes?",
    "How to implement Interfaces in Python?",
]

for question in questions:
    print(f"[b]Question:[/b] [i green]{question}[/i green]")
    output = full_chain.invoke(question)
    display(Markdown(output))
    print()
    print("---" * 25)

Question: List last 5 presidents of USA and their term durations. Output in a Markdown table format.

 Here is a table listing the last 5 presidents of the USA and their term durations:

| President | Term |
|-|-|  
| Joe Biden | 2021 - present |
| Donald Trump | 2017 - 2021 |
| Barack Obama | 2009 - 2017 | 
| George W. Bush | 2001 - 2009 |
| Bill Clinton | 1993 - 2001 |

---------------------------------------------------------------------------

Question: What is 5% of 257

 Okay, let's break this down step-by-step:

1) 5% means 5 out of 100
2) To find 5% of a number, we calculate:
(5 / 100) * 257

3) (5 / 100) = 0.05

4) 0.05 * 257 = 12.85

Therefore, 5% of 257 is 12.85.

...

 Black holes are regions of spacetime exhibiting gravitational acceleration so strong that nothing—no particles or even electromagnetic radiation such as light—can escape from inside it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. The boundary of no escape is called the event horizon. Although it has a great effect on the fate and circumstances of an object crossing it, it has no locally detectable features according to general relativity. In many ways, a black hole acts like an ideal black body, as it reflects no light. Moreover, quantum field theory in curved spacetime predicts that event horizons emit Hawking radiation, with the same spectrum as a black body of a temperature inversely proportional to its mass. This temperature is extremely low for stellar black holes, making it essentially impossible to observe directly.

---------------------------------------------------------------------------

Question: How to implement Interfaces in Python?

 Python does not have an interface construct like other object-oriented languages such as Java or C#. However, Python does allow for interfaces to be implemented through abstract base classes. Here is one way to implement interfaces in Python:

1. Create an abstract base class and use the abc module to make it an official abstract class:

```python
from abc import ABC, abstractmethod

class MyInterface(ABC):

    @abstractmethod
    def method1(self):
        pass

    @abstractmethod    
    def method2(self):
        pass
```

2. Any class that inherits from this abstract base class must implement all the abstract methods: 

```python 
class MyClass(MyInterface):

    def method1(self):
        print("Implementing method1")

    def method2(self):
        print("Implementing method2")
```

3. Attempting to instantiate the abstract base class directly will raise an error.

4. The abstract base class can be used to check if a class implements the interface:

```python
def func(arg):
    if isinstance(arg, MyInterface):
        # Interface has been implemented
        arg.method1()
        arg.method2()
```

So in summary, interfaces can be implemented in Python by creating abstract base classes that define abstract methods that any subclass must implement.

---------------------------------------------------------------------------

### Invoking with `RunnableLambda`

If you prefer to implement your own Lambda as a routing logic you can do so like below:

In [15]:
def route_prompt(input):
    if "math" in input["topic"].lower():
        return math_chain
    elif "physics" in input["topic"].lower():
        return physics_chain
    elif "history" in input["topic"].lower():
        return history_chain
    elif "computerscience" in input["topic"].lower():
        return computerscience_chain
    else:
        return general_chain

In [16]:
full_chain = {
    "topic": classify_chain,
    "question": RunnablePassthrough(),
} | RunnableLambda(route_prompt)

In [17]:
questions = [
    "What transpired during the battle of Gettsyburg.",
    "What is 55% of 7Billion",
    "What is Max Planck's theory on consciousness?",
    "What are abstract classes in Python?",
]

for question in questions:
    print(f"[b]Question:[/b] [i green]{question}[/i green]")
    output = full_chain.invoke(question)
    display(Markdown(output))
    print()
    print("---" * 25)

Question: What transpired during the battle of Gettsyburg.

 Here is a summary of what transpired during the Battle of Gettysburg in the American Civil War:

- The battle took place from July 1-3, 1863 in and around the town of Gettysburg, Pennsylvania. It was a major turning point in the war.

- On July 1, Confederate forces under General Robert E. Lee engaged Union cavalry under General John Buford outside Gettysburg. Despite being outnumbered, Buford's men delayed the Confederate advance until Union reinforcements arrived.  

- On July 2, Lee launched assaults on the Union flanks. Fighting raged at locations such as the Peach Orchard, the Wheatfield, and Devil's Den. The Union lines generally held despite heavy casualties. 

- On July 3, Lee ordered what became known as Pickett's Charge, a massive frontal assault on the center of the Union lines on Cemetery Ridge. After an intense artillery bombardment, around 12,500 Confederate soldiers advanced but were repulsed with heavy losses. 

- Lee's defeat ended his invasion of the North. The Union had held important ground and inflicted heavy losses on the Confederates, around 28,000 men. It was a major turning point that contributed to the eventual Confederate surrender in 1865.

- Gettysburg also saw President Lincoln deliver his famous Gettysburg Address, dedicating the battlefield to the soldiers who died there.

---------------------------------------------------------------------------

Question: What is 55% of 7Billion

 Okay, let's break this down step-by-step:

1) 7 billion can be written as 7,000,000,000

2) 55% of something is the same as 0.55 * that something

3) So 55% of 7,000,000,000 is:
0.55 * 7,000,000,000

4) Multiplying this out:
0.55 * 7,000,000,000 = 3,850,000,000

Therefore, the answer is:
55% of 7 billion is 3,850,000,000

...

 Unfortunately I do not have enough information to provide a substantive answer about Max Planck's views on consciousness. Max Planck was a pioneering German physicist who made fundamental contributions to quantum theory, but he did not extensively discuss theories of consciousness. If you have a more specific question about Planck's scientific work or views, I would be happy to try to answer.

---------------------------------------------------------------------------

Question: What are abstract classes in Python?

 Abstract classes in Python are classes that contain one or more abstract methods. An abstract method is a method that is declared, but contains no implementation. Abstract classes cannot be instantiated, and require subclasses to provide implementations for the abstract methods. 

Here are the key characteristics of abstract classes and abstract methods in Python:

- Abstract classes are defined by inheriting from the ABC (Abstract Base Class) module's ABC class. 

- Abstract methods are declared with the @abstractmethod decorator.

- Any class that contains abstract methods must also be declared abstract.

- Abstract classes cannot be instantiated.

- To use an abstract class, it must be subclassed, and the subclasses must provide implementations for all abstract methods.

- Abstract methods do not contain implementation - they only have the method signature and optional docstring.

- Abstract classes allow related classes to share a common interface and enforce certain methods, properties and behaviors.

So in summary, abstract classes define a common interface for their subclasses and contain abstract methods that the subclasses must implement. This allows related classes to share a common abstraction while deferring the implementation of certain methods to the subclasses.

---------------------------------------------------------------------------

---

### [Legacy] Router Chain (OPTIONAL)

**[!WARNING]**
_THE PREFERRED APPROACH AS OF VERSION 0.0.293 IS TO USE LCEL AS ABOVE._

Reference: <https://python.langchain.com/docs/modules/chains/foundational/router>

In [18]:
# physics_template = """You are a very smart physics professor. \
# You are great at answering questions about physics in a concise\
# and easy to understand manner. \
# When you don't know the answer to a question you admit\
# that you don't know.

# Here is a question:
# {input}"""


# math_template = """You are a very good mathematician. \
# You are great at answering math questions. \
# You are so good because you are able to break down \
# hard problems into their component parts,
# answer the component parts, and then put them together\
# to answer the broader question.

# Here is a question:
# {input}"""

# history_template = """You are a very good historian. \
# You have an excellent knowledge of and understanding of people,\
# events and contexts from a range of historical periods. \
# You have the ability to think, reflect, debate, discuss and \
# evaluate the past. You have a respect for historical evidence\
# and the ability to make use of it to support your explanations \
# and judgements.

# Here is a question:
# {input}"""


# computerscience_template = """ You are a successful computer scientist.\
# You have a passion for creativity, collaboration,\
# forward-thinking, confidence, strong problem-solving capabilities,\
# understanding of theories and algorithms, and excellent communication \
# skills. You are great at answering coding questions. \
# You are so good because you know how to solve a problem by \
# describing the solution in imperative steps \
# that a machine can easily interpret and you know how to \
# choose a solution that has a good balance between \
# time complexity and space complexity.

# Here is a question:
# {input}"""

In [19]:
# prompt_infos = [
#     {
#         "name": "physics",
#         "description": "Good for answering questions about physics",
#         "prompt_template": physics_template,
#     },
#     {
#         "name": "math",
#         "description": "Good for answering math questions",
#         "prompt_template": math_template,
#     },
#     {
#         "name": "History",
#         "description": "Good for answering history questions",
#         "prompt_template": history_template,
#     },
#     {
#         "name": "computer science",
#         "description": "Good for answering computer science questions",
#         "prompt_template": computerscience_template,
#     },
# ]

In [20]:
# destination_chains = {}
# for p_info in prompt_infos:
#     name = p_info["name"]
#     prompt_template = p_info["prompt_template"]
#     prompt = ChatPromptTemplate.from_template(template=prompt_template)
#     chain = LLMChain(llm=llm_instant, prompt=prompt)
#     destination_chains[name] = chain

# destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# destinations_str = "\n".join(destinations)

#### Define a default_chain for inputs that doesn't match with any of the provided template.

In [21]:
# default_prompt = ChatPromptTemplate.from_template("{input}")
# default_chain = LLMChain(llm=llm_instant, prompt=default_prompt)

#### Define the Multi-prompt Router prompt

In [22]:
# MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
# language model select the model prompt best suited for the input. \
# You will be given the names of the available prompts and a \
# description of what the prompt is best suited for. \
# You may also revise the original input if you think that revising \
# it will ultimately lead to a better response from the language model.

# << FORMATTING >>
# Return a markdown code snippet with a JSON object formatted to look like:
# ```json
# {{{{
#     "destination": string \ name of the prompt to use or "DEFAULT"
#     "next_inputs": string \ a potentially modified version of the original input
# }}}}
# ```

# REMEMBER: "destination" MUST be one of the candidate prompt \
# names specified below OR it can be "DEFAULT" if the input is not\
# well suited for any of the candidate prompts.
# REMEMBER: "next_inputs" can just be the original input \
# if you don't think any modifications are needed.

# << CANDIDATE PROMPTS >>
# {destinations}

# << INPUT >>
# {{input}}

# << OUTPUT (remember to include the ```json)>>"""

In [23]:
# router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# router_prompt = PromptTemplate(
#     template=router_template,
#     input_variables=["input"],
#     output_parser=RouterOutputParser(),
# )

# router_chain = LLMRouterChain.from_llm(llm_instant, router_prompt)

In [24]:
# chain = MultiPromptChain(
#     router_chain=router_chain,
#     destination_chains=destination_chains,
#     default_chain=default_chain,
#     verbose=True,
# )

In [25]:
# output = chain.run(
#     "List last 5 presidents of USA and their term durations. Output in a Markdown table format."
# )

# Markdown(output)